---
title: Substitution Etudes
date: 2025-06-09
---

Examining the categorical formulation of simple substitutions is useful to understand more complex topics.

Simple substitutions form a category.

The objects are ordered lists of sorts representing the variables in contexts `[smt.Int("x"), smt.Real("y"), ...]`.

Substitutions compose and have an identity in a natural way

We give them names to easily reference them. Everything is actually alpha equivalent with respect to the names.
`ts` are expressions like `[cos(x), sin(cos(x))]`. They kind of match a naive notion of "function expression".

You can apply these things to a list of terms and they'll inject those terms into the variable positions.


In [ ]:
from dataclasses import dataclass
from kdrag.all import *
@dataclass
class Subst():
    dom : list[smt.ExprRef] # a list of variables
    cod : list[smt.ExprRef] # Kind of redudnant but useful for seeing connection to telescopes
    ts : list[smt.ExprRef] # same length and types as cod, with vars from dom
    def __post_init__(self):
        assert len(self.cod) == len(self.ts)
    def __call__(self, *args):
        """ Apply subst to multi term"""
        if len(args) != len(self.dom):
            raise ValueError(f"Expected {len(self.dom)} arguments, got {len(args)}")
        return [smt.substitute(self.ts, *zip(self.dom, args)) for t in self.ts]
    def __matmul__(self, other):
        """Composition plugs other's terms into self's holes"""
        assert isinstance(other, Subst)
        assert len(self.dom) == len(other.cod) and all(v.sort() == w.sort() for v, w in zip(self.dom, other.cod))
        return Subst(other.dom, self.cod, [smt.substitute(t, *zip(self.dom, other.ts)) for t in self.ts])
    @classmethod
    def id_(cls, xs : list[smt.ExprRef]):
        """ identity map does nothing"""
        return Subst(xs, xs, xs)
    def __mul__(self, other : Subst):
        # a monoidal product of substitutions. Doing them in parallel.
        assert isinstance(other, Subst)
        def freshen(xs):
            return [smt.FreshConst(x.sort(),prefix=x.decl().name()) for x in xs]
        odom = freshen(other.dom)
        ocod = freshen(other.cod)
        ots = [smt.substitute(t, *zip(other.dom, odom)) for t in other.ts]
        return Subst(self.dom + odom, self.cod + ocod, self.ts + odom)
    def __eq__(self, other):
        """The proposition that two substitutions/morphisms are equal. To be assumed or proven"""
        assert len(self.dom) == len(other.dom) and len(self.cod) == len(other.cod)
        assert all(v.sort() == w.sort() for v, w in zip(self.dom, other.dom))
        assert all(v.sort() == w.sort() for v, w in zip(self.cod, other.cod))
        ots = [smt.substitute(t, *zip(other.dom, self.dom)) for t in other.ts] # rename to common variables self.dom
        return smt.ForAll(self.dom, smt.And([t1 == t2 for (t1,t2) in zip(self.ts, ots)]))


x,y,z = smt.Ints('x y z')
Subst.id_([x]) @ Subst.id_([y])



Subst(dom=[y], cod=[x], ts=[y])

In [30]:
Subst([x], [y,z], [x + 1, x - 2]) @ Subst([x], [y], [x * 7])


Subst(dom=[x], cod=[y, z], ts=[x*7 + 1, x*7 - 2])

In [31]:

Subst.id_([x]) * Subst.id_([y])

Subst(dom=[x, y!7], cod=[x, y!8], ts=[x, y!7])

A sequence of ground terms can be lifted into a subst coming from the empty context. This is a standard categorical trick of pointing out a particular thing by using a map from some unit object.

In [ ]:
def lift_ground(*ts) -> Subst:
    return Subst([], [smt.FreshConst(t.sort()) for t in ts], list(ts))

Function symbols can be seen as the substitution that would apply them

In [ ]:
def lift_funcdecl(f : smt.FuncDeclRef):
    args = [smt.FreshConst(f.domain(n)) for n in range(f.arity())]
    out = [smt.FreshConst(f.range())]
    return Subst(args, out, [f(*args)])

foo = smt.Function("foo", smt.IntSort(), smt.RealSort(), smt.BoolSort())
lift_funcdecl(foo)

The is an opposite category of substitutions that instead of applying the `ts` to `args`, narrows the the variables in a predicate P(x) to the ts. Considering the predicate as a morphism from the vars to `[Bool]`, this is reverse composition.

P can also be considered as a Subst to Bool.

In [ ]:
def subst(s : Subst, P : smt.ExprRef) -> smt.ExprRef: 
    return smt.substitute(P, zip(s.cod, s.ts))

def lift_pred(vs : list[smt.ExprRef], P : smt.ExprRef) -> Subst:
    """
    Lift a predicate P over variables vs to a substitution.
    """
    return Subst(vs, [smt.FreshConst(P.sort())], [P])


What is a unification problem in these terms?

It is useful to extend the notion of a single equation to a multiequation.
Then what we have is two substitutions need to be equal.

```
          - S >  
k - H > m        n
          - T > 
```

For some reason literature seems to describe unification as as coequalizer. I don't understand why. Maybe I'm totally wrong. Or perhaps that are taking the "narrowing" interpretation of substitutions rather than the "applying" interpretation.

See Goguen What is Unification https://www.sciencedirect.com/science/article/abs/pii/B9780120463701500127?via%3Dihub https://www.cs.bu.edu/fac/snyder/publications/UnifChapter.pdf  section 3.3.3 

In [ ]:
def unify(s : Subst, t : Subst):
    assert len(s.dom) == len(t.dom) and all(v.sort() == w.sort() for v, w in zip(s.dom, t.dom))
    assert len(s.cod) == len(t.cod) and all(v.sort() == w.sort() for v, w in zip(s.cod, t.cod))
    todo = list(zip(s.ts, [smt.substitute(e, *zip(t.dom, s.dom)) for e in t.ts]))
    subst = Subst.id_(s.dom) # null substitution
    while todo:
        (s,t) = todo.pop()
        if s in s.dom: # is var
            newsubst = ...
            subst = subst @ newsubst
        #yada. Not sure it's calirfying to write this in this way
    

Equational axioms can be expressions as the commuting squares of certain arrows.
https://ncatlab.org/nlab/show/Lawvere+theory
https://en.wikipedia.org/wiki/Lawvere_theory

Every object is a product of a distinguished object: yes.
Homomorphisms are functors in this context. This is nice tight formulation of homomorphism, although I find the more expanded definition much more intuitive. https://en.wikipedia.org/wiki/Homomorphism

In [64]:
add  = Subst([x,y], [z], [x + y])
swap = Subst([x,y], [y,x], [y,x])
kd.prove(add == add @ swap)

|- ForAll([x, y], And(x + y == y + x))

## Telescopes

An interesting thing to do is start enriching the notion of object / context / things substitions go from and to.

One formulation of telescopes https://ncatlab.org/nlab/show/type+telescope for a refinement type looking thing is to pack every variable with a boolean expression describing what preconditions / subset the subtitution is coming from. These are partial substitutions in some sense.

The meaning of a telescope can be somewhat explained by showing how to "forall" a predicate in that context using TForAll. Because of the iterated construction, the earlier variables are in scope for the later variables.



In [3]:
from kdrag.all import *
type Tele = list[tuple[smt.ExprRef, smt.BoolRef]]
def TForAll(vs : Tele, P):
    for v,pre in reversed(vs):
        P = smt.ForAll([v], smt.Implies(pre, P))
    return P

x,y,z = smt.Ints('x y z')
TForAll([(x, x > 0), (y, y > 0)], x + y > 0)


ForAll(x,
       Implies(x > 0, ForAll(y, Implies(y > 0, x + y > 0))))

Telescope mappings are a generalization of substitutions. We can use them to organize a theorem proving process involving pre and post conditions.
They represent a theorem 


In [4]:
from dataclasses import dataclass
@dataclass
class TeleMap():
    dom : Tele
    cod : Tele
    ts : list[smt.ExprRef]  # len(cod) expression using variables in dom. When dom preconditions are true, cod conditions should hold of ts.
    def __post_init__(self):
        assert len(self.ts) == len(self.cod)
        subst = [(v, t) for (v, _), t in zip(self.cod, self.ts)]
        self.pfs = []
        for (v, cond) in self.cod:
            self.pfs.append(kd.prove(TForAll(self.dom, smt.substitute(cond, *subst))))
    def __call__(self, *args):
        assert len(args) == len(self.dom)
        subst = [(v, a) for (v, _), a in zip(self.dom, args)]
        #for pf in self.pfs:
        for (_, pre) in self.dom:
            kd.prove(smt.substitute(pre, *subst))
        return [smt.substitute(t, *subst) for t in self.ts]
    @classmethod
    def id_(cls, xs : Tele):
        return cls(xs, xs, [v for v, _ in xs])
    def __matmul__(self, other):
        assert len(self.dom) == len(other.cod)
        subst = [(v, w) for (v, _), (w, _) in zip(self.dom, other.cod)]
        ws = [w for w,_ in other.cod]
        vs = [v for v,_ in self.dom]
        for (v, pre), (w, post) in zip(self.dom, other.cod):
            assert v.sort() == w.sort(), f"Sort mismatch: {v.sort()} != {w.sort()}"
            # allows subset composition rather than on the nose composition.
            kd.prove(TForAll(other.cod, smt.substitute(pre, *subst))) 
        return TeleMap(other.dom, self.cod, [smt.substitute(t, *zip(vs, other.ts)) for t in self.ts])

TeleMap([(x, x > -1)],[(y, y > 0)], [x + 1])  @ TeleMap([(x, x > 0)], [(y, y > 0)], [x + 1])

TeleMap(dom=[(x, x > 0)], cod=[(y, y > 0)], ts=[x + 1 + 1])

A well formed telescope mapping represents a theorem about the mappings in the `ts`. If all variables satisfy the preconditions in the `self.dom` telescope, the terms `self.ts` should obey the postconditions in the `self.cod`

In [69]:
TeleMap([(x, x > 0), (z, z > x)], [(y, y > 0), (z, z > 1)], [x + z, z]).pfs

[|- ForAll(x,
        Implies(x > 0, ForAll(z, Implies(z > x, x + z > 0)))),
 |- ForAll(x, Implies(x > 0, ForAll(z, Implies(z > x, z > 1))))]

In [44]:
TeleMap([(x, x > 0)], [(y, y > 0)], [x + 1])(smt.IntVal(7))

[7 + 1]

In [ ]:
# fails to meet precondition
TeleMap([(x, x > 0)], [(y, y > 0)], [x + 1])(smt.IntVal(-2))

LemmaError: (-2 > 0, 'Countermodel', [])

In [9]:
x = smt.Bool("x")
TRUE : Tele = [(x, x)]
TRUE2 = TeleMap([], TRUE, [smt.BoolVal(True)])
Bool = [(x, smt.BoolVal(True))]
FALSE : Tele = [(x, smt.Not(x))]
smt.Int("y")
ZERO = TeleMap([(y, y == 0)], TRUE, [y == 0])
POS = TeleMap([(y, y > 0)], TRUE, [y > -1])

# Bits and Bobbles

Yea, I dunno. Not what I set out to write today. not sure what we've learned.

(x, x)


Telescope unification?

The equalizer in a lawvere theory is E-unification.

coalgebra.
a -> f a

operads are term rewriting equations with linearity on the rules 

GATs

EATs

https://people.mpi-sws.org/~dreyer/courses/catlogic/jacobs.pdf

https://ncatlab.org/nlab/files/HofmannExtensionalIntensionalTypeTheory.pdf Amongst the many nice things in this thesis are a good description of telescopes on p. 26 section 2.2.2
`Gamma |- f => Delta` 

https://proofassistants.stackexchange.com/questions/755/whats-the-relationship-between-refinement-types-and-dependent-types


Hmm. Maybe explicit cod was a mistake. I dunno.

I am puzzled about whether there is really that much utility of the telescope form `forall x, p(x) => forall y, q(x,y) => ...` vs just the smashed together version `forall x y ..., p(x) /\ q(x,y) /\ ... `. The latter is more reminiscent of Isabelle Pure.


https://math.andrej.com/2012/09/28/substitution-is-pullback/ 

https://www.cs.bu.edu/fac/snyder/publications/UnifChapter.pdf

# Subset Model
https://www.philipzucker.com/frozenset_dtt/ Showed how to use finite sets the interpret some dependent types


In [ ]:
from typing import Callable
from frozendict import frozendict
import itertools
type Family = Callable[[object], Type]
type Type = frozenset

Void = frozenset({})
Unit = frozenset({()})
Bool = frozenset({True, False})
def Fin(n : int) -> Type:
    return frozenset(range(n))
def Vec(A : Type, n : int) -> Type:
    return frozenset(itertools.product(A, repeat=n))

def is_type(A: Type) -> bool: # |- A type
    return isinstance(A, frozenset)
def has_type(t: object, A: Type) -> bool: # |- t : A
    return t in A
def eq_type(A: Type, B: Type) -> bool: # |- A = B type
    return A == B
def def_eq(x : object, y: object, A : Type) -> bool: # |- x = y : A
    return x == y and has_type(x, A) and has_type(y, A)

def Sigma(A: Type, B: Family) -> Type:
    return frozenset({(a, b) for a in A for b in B(a)})
def Pair(A : Type, B: Type) -> Type:
    return Sigma(A, lambda x: B)

def Pi(A : Type, B : Family) -> Type:
    Alist = list(A)
    return frozenset(frozendict({k:v for k,v in zip(Alist, bvs)}) for bvs in itertools.product(*[B(a) for a in Alist]))
def Arr(A : Type, B: Type) -> Type:
    return Pi(A, lambda x: B)

def Sum(A : Type, B: Type) -> Type:
    return frozenset({("inl", a) for a in A} | {("inr", b) for b in B})

def Id(A : Type, x : object, y : object) -> Type:
    return frozenset({"refl"}) if x == y else frozenset()
def U(n : int, l : int) -> Type:
    if l > 0:
        u = U(n, l-1)
        return u | frozenset([u]) # Cumulative
    elif n > 0:
        u = U(n-1, 0)
        # TODO also the Pi and Sigma
        return u | frozenset([Arr(A,B) for A in u for B in u]) | frozenset([Pair(A,B) for A in u for B in u]) | frozenset([Fin(n)])
    else:
        return frozenset([Unit, Bool, Void])
def Quot(A : Type, R) -> Type:
    return frozenset(frozenset({y for y in A if R(x,y)}) for x in A)
def SubSet(A :  Type, P : Family) -> Type: # very much like Sigma
    return frozenset({(x, ()) for x in A if P(x)}) # Note because of pythion truthiness, this also accepts ordinary bool value predicates.


We can also make a subset model as mentioned here https://ncatlab.org/nlab/files/HofmannExtensionalIntensionalTypeTheory.pdf 

Types are interpreted as a pair of a frozenset and a callable function representing the subset orf that frozenset


There are now two equalities to talk about. On the nose equality `==` and kleene equality
https://en.wikipedia.org/wiki/Kleene_equality



In [ ]:
from typing import Callable
from frozendict import frozendict
import itertools
type Type = tuple[frozenset, Callable[[object], bool]]
type Family = tuple[Callable[[object], frozenset], Callable[[object], Callable[[object], bool]]]
# type Family = Callable[[object], Type] # A family of types, indexed by an object
TRUE = lambda: True
FALSE = lambda: False

Void = (frozenset({}), TRUE)
Unit = (frozenset({()}), TRUE)
Bool = (frozenset({True, False}), TRUE)
def Fin(n : int) -> Type:
    return (frozenset(range(n)), TRUE)
def Sigma(A: Type, B: Family) -> Type:
    return frozenset({(a, b) for a in A[1] for b in B[0](a)}), lambda ab: B[1](ab[0])(ab[1]) if A[1](ab[0]) else FALSE
def def_eq(x, y, A):
    return x == y
def ext_eq(x,y,A): # kleene equality https://en.wikipedia.org/wiki/Kleene_equality
    return (not A[1](x) and not A[1](y)) or x == y #??? not so sure
